In [23]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [24]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [25]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [26]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'ATVI', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC',
       'ESAB', 'FCFS', 'GEN', 'IBKR', 'ICPT', 'JJC', 'NATI', 'PEAK', 'PFG',
       'SBNY', 'SRPT', 'TWNK', 'VC', 'VMW', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [27]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ABM', 'ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG',
       'AMGN',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=247)


In [28]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'ALGM', 'ATVI', 'BJ', 'BRBR', 'CARR', 'CDAY', 'CEG',
       'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG', 'DOW', 'DT', 'DTM',
       'ESAB', 'ETRN', 'FOX', 'FOXA', 'FYBR', 'GEHC', 'GFS', 'GO', 'GXO',
       'HEP', 'ICPT', 'JJC', 'KD', 'LCID', 'MP', 'MRNA', 'NARI', 'NATI', 'NCR',
       'NVST', 'NVT', 'OGN', 'OSTK', 'OTIS', 'PACW', 'PDD', 'PGNY', 'RIVN',
       'SGOV', 'SHC', 'SITM', 'SWAV', 'TWNK', 'VICI', 'VMW', 'VNT', 'VSCO',
       'WH', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [29]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 306


In [30]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-12-06,120,"[30, 60, 120]",[],[],[],[],[],[],[],"['GPS', 'SHV']","['FTSM', 'VRNS']","['LRN', 'WING']","['ANF', 'CBOE', 'GBTC', 'MLKN']","['BTC-USD', 'GDDY', 'HIBB', 'WSM']","['BPMC', 'EDU', 'EFX', 'HA', 'NRG', 'SPR']",[]


In [31]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],[],"['GPS', 'SHV']","['FTSM', 'VRNS']","['LRN', 'WING']","['ANF', 'CBOE', 'GBTC', 'MLKN']","['BTC-USD', 'GDDY', 'HIBB', 'WSM']","['BPMC', 'EDU', 'EFX', 'HA', 'NRG', 'SPR']",[]


In [32]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [33]:
import ast

In [34]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 6400
unique symbol count from model picks: 316


In [35]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [36]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['CHT', 'CMG', 'CVBF', 'CVLT', 'DVA', 'HEP', 'HOLI', 'HY', 'ICPT', 'MLR', 'NATI', 'OSIS', 'OSTK', 'PKX', 'SFBS', 'SRPT', 'TWNK', 'VNO', 'XENE']


In [37]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [38]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 306
len(common_symbols): 19
len(symbols_drop): 287


In [39]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 287


In [40]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      26
symbols with same volume:   247
symbols with no close:       60

symbols total before drop:                                        1,557
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   287

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,270    1,500
df Close (df_c) after dropped symbols:      1,270    1,500


In [41]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [42]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [43]:
df_a.tail()

A                                                 \
                  Open        High         Low       Close     Volume   
Date                                                                    
2023-12-05  127.809998  128.630005  125.769997  127.879997  1933600.0   
2023-12-06  128.500000  129.759995  127.919998  128.910004  1816800.0   
2023-12-07  129.199997  130.479996  127.809998  128.679993  1696600.0   
2023-12-08  128.089996  129.000000  126.970001  127.199997  1438100.0   
2023-12-11  127.620003  129.630005  127.220001  128.970001  1879251.0   

                   AA                                              ...  \
                 Open       High        Low      Close     Volume  ...   
Date                                                               ...   
2023-12-05  26.500000  26.530001  25.129999  25.240000  7715800.0  ...   
2023-12-06  25.600000  25.959999  25.040001  25.120001  4055000.0  ...   
2023-12-07  25.160000  25.290001  24.270000  24.940001  6780000.0  ...   
2023-12-08  25.040001  25.725000  24.870001  25.020000  5128100.0  ...   
2023-12-11  24.750000  25.319901  24.670000  24.930000  4167712.0  ...   

                 ZUMZ                                                   ZWS  \
                 Open       High        Low      Close    Volume       Open   
Date                                                                          
2023-12-05  19.920000  19.920000  19.520000  19.600000  220500.0  29.770000   
2023-12-06  19.900000  19.990000  19.370001  19.430000  258100.0  29.440001   
2023-12-07  19.350000  19.530001  19.250000  19.469999  231400.0  29.219999   
2023-12-08  19.459999  19.490000  19.049999  19.350000  207800.0  29.440001   
2023-12-11  19.480000  19.588699  18.860001  18.969999  289688.0  29.350000   

                                                       
                 High        Low      Close    Volume  
Date                                                   
2023-12-05  29.959999  29.250000  29.299999  927400.0  
2023-12-06  29.780001  29.190001  29.209999  951300.0  
2023-12-07  29.540001  29.080000  29.480000  676600.0  
2023-12-08  29.809999  29.299999  29.389999  729700.0  
2023-12-11  29.749500  29.290001  29.610001  815705.0  

[5 rows x 6350 columns]

In [44]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABR,ABT,ACGL,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-12-05,127.879997,25.240000,13.13,53.160000,193.419998,28.430000,144.570007,13.67,104.349998,81.849998,...,37.669998,115.820000,234.360001,62.380001,42.520000,37.389999,21.389999,179.649994,19.600000,29.299999
2023-12-06,128.910004,25.120001,13.48,55.160000,192.320007,28.420000,146.380005,13.15,104.940002,79.970001,...,37.520000,116.129997,234.130005,62.750000,43.490002,37.150002,21.420000,182.000000,19.430000,29.209999
2023-12-07,128.679993,24.940001,13.91,56.250000,194.270004,29.400000,147.970001,13.64,104.050003,78.339996,...,37.730000,117.290001,233.229996,63.119999,45.009998,38.380001,21.260000,181.830002,19.469999,29.480000
2023-12-08,127.199997,25.020000,13.76,56.270000,195.710007,30.030001,149.279999,13.65,104.510002,79.300003,...,37.439999,117.300003,236.130005,63.250000,44.660000,38.930000,21.309999,184.600006,19.350000,29.389999
2023-12-11,128.970001,24.930000,13.72,57.080002,193.179993,29.820000,151.240005,14.10,106.220001,79.610001,...,36.939999,117.980003,239.649994,63.730000,43.820000,38.630001,21.020000,189.460007,18.969999,29.610001
